<a href="https://colab.research.google.com/github/impos0108/AI_basic_for_atmospheric_science/blob/main/CNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 43659766.54it/s]


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 61056297.29it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33812091.56it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14302198.77it/s]


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [4]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [5]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

In [6]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [7]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.1690
Epoch [1/5], Step [200/600], Loss: 0.1359
Epoch [1/5], Step [300/600], Loss: 0.1232
Epoch [1/5], Step [400/600], Loss: 0.0359
Epoch [1/5], Step [500/600], Loss: 0.2178
Epoch [1/5], Step [600/600], Loss: 0.0593
Epoch [2/5], Step [100/600], Loss: 0.0356
Epoch [2/5], Step [200/600], Loss: 0.1630
Epoch [2/5], Step [300/600], Loss: 0.0323
Epoch [2/5], Step [400/600], Loss: 0.0212
Epoch [2/5], Step [500/600], Loss: 0.0197
Epoch [2/5], Step [600/600], Loss: 0.0188
Epoch [3/5], Step [100/600], Loss: 0.0229
Epoch [3/5], Step [200/600], Loss: 0.0024
Epoch [3/5], Step [300/600], Loss: 0.0203
Epoch [3/5], Step [400/600], Loss: 0.0360
Epoch [3/5], Step [500/600], Loss: 0.0236
Epoch [3/5], Step [600/600], Loss: 0.1107
Epoch [4/5], Step [100/600], Loss: 0.0517
Epoch [4/5], Step [200/600], Loss: 0.0048
Epoch [4/5], Step [300/600], Loss: 0.0283
Epoch [4/5], Step [400/600], Loss: 0.0205
Epoch [4/5], Step [500/600], Loss: 0.0035
Epoch [4/5], Step [600/600], Loss:

In [8]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 98.53 %


In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'cnn_model.ckpt')